In [ ]:
import json
import requests

root_url = "http://localhost:8000/"

### Create a user

In [ ]:
user1_data = {
	"@id": "ark:99999/test-user1",
	"name": "Test User1",
	"type": "Person",
	"email": "testuser1@example.org",
	"password": "test1",
	"organizations": [],
	"projects": [],
	"datasets": [],
	"software": [],
	"computations": [],
	"evidencegraphs": []
	}

# create user1
create_user1 = requests.post(root_url + "user", data=json.dumps(user1_data))
create_user1.json()

### Create a Software

In [ ]:
# create software
software_data = {
	"@id": "ark:99999/test-software",
	"@type": "evi:Software",
	"name": "test-software",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}#,
	#"author": "some author(s)",
    #"downloadUrl": "some url",
    #"citation": "some citation",
	#"evi:usedBy": []
}

software_create = requests.post(root_url + f"software/", data=json.dumps(software_data))
software_create.json()

### Create a Dataset

In [ ]:
# create dataset
dataset_data = {
	"@id": "ark:99999/test-dataset",
	"@type": "evi:Dataset",
	"name": "test-dataset",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
}

dataset_create = requests.post(root_url + f"dataset/", data=json.dumps(dataset_data))
dataset_create.json()

### Create a Computation

In [ ]:
# create computation
computation_data = {
	"@id": "ark:99999/test-computation",
	"@type": "evi:Computation",
	"name": "test computation",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}#,
	# "author" : "",
    #"dateCreated" : "",
    #"dateFinished" : "",
    # "associatedWith" : [],
    # "usedSoftware" : "",
    # "usedDataset" : ""
}

computation_create = requests.post(root_url + f"computation/", data=json.dumps(computation_data))
computation_create.json()

### Create metadata for the dataset to be uploaded

In [ ]:
# create DataDownload metadata for a Dataset
test_data_download_for_dataset = {
	"@id": "ark:99999/test-download-dataset",
	"name": "random_numbers.txt",
	"@type": "DataDownload",
	"encodingFormat": ".txt",
	"encodesCreativeWork": dataset_data["@id"]
}

data_download_create = requests.post(root_url + f"datadownload", data=json.dumps(test_data_download_for_dataset))
data_download_create.json()

### Upload the dataset

In [ ]:
# upload a dataset to minio object store
data_files = {'file': ('random_numbers.txt', '1\n2\n3\n4\n5')}
data_download_upload = requests.post(root_url + f"datadownload/ark:99999/test-download-dataset/upload", files=data_files)
data_download_upload.json()

In [ ]:
# read the DataDownload metadata for the dataset
data_download_read = requests.get(root_url + f"datadownload/ark:99999/test-download-dataset")
data_download_read.json()

### Create metadata for the script to be uploaded

In [ ]:
# create DataDownload metadata for a Software
test_data_download_for_software = {
	"@id": "ark:99999/test-download-software",
	"name": "sum_script.py",
	"@type": "DataDownload",
	"encodingFormat": ".py",
	"encodesCreativeWork": software_data["@id"]
}

data_download_create = requests.post(root_url + f"datadownload", data=json.dumps(test_data_download_for_software))
data_download_create.json()

### Upload the script (Required: Create 'input-script' directory inside 'SOURCE_VOL' and store 'sum_script.py' for this operation)

In [ ]:
# upload a software to minio object store
script_file = {'file': open('/home/sadnan/compute-test/input-script/sum_script.py', 'r')}
data_download_upload = requests.post(root_url + f"datadownload/ark:99999/test-download-software/upload", files=script_file)
data_download_upload.json()

In [ ]:
# read the DataDownload metadata for the dataset
data_download_read = requests.get(root_url + f"datadownload/ark:99999/test-download-software")
data_download_read.json()

# Perform computation by running the script on the dataset

In [ ]:

compute_resources = {
	"datasetID": ["ark:99999/test-dataset"],
	"scriptID": "ark:99999/test-software"
}

compute_run = requests.post(root_url + f"compute/ark:99999/test-computation", json.dumps(compute_resources))
compute_run.json()

### Metadata for the Computation have been updated

In [ ]:
get_computation = requests.get(root_url + f"computation/ark:99999/test-computation")
get_computation.json()